 # <h2><center><strong style="color:#ff6161"> Detecting Linux system hacking activities Part B
 </center></strong></h3>
 

<h2> 1. Producing the data 

<h3> 1.1 Process Event Producer

In [ ]:
# Import libraries and initializing 

from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime
from csv import DictReader


In [ ]:
#Function to publish producer messages to the topic queue 

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [ ]:
#Function to connect to the Kafka Producer

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [ ]:
#Function to add timestamp to the data
def add_timestamp(data_rows,tst):
    data = []
    #ts = datetime.datetime.utcnow().strftime("%s")
    for row in data_rows:
        row['ts'] = tst
        data.append({k: row[k] for k in row.keys()})
    return data_rows

In [ ]:
#Function to alter the key values from string to integer and float values
def alter_values(data_list):
    for sub in data_list:
        for key in sub:
            try:
                sub[key] = int(sub[key]) 
                sub['CPU']= float(sub[key]) 
            except ValueError:
                continue
    return data_list

In [ ]:
#Initializing the variables used in loop
ts = 0
machine_4 = []
machine_5 = []
machine_6 = []
machine_7 = []
machine_8 = []

#Initializing the producer instance
producer = connect_kafka_producer()

#Initializing the process topic name
topic = 'processaa12'

#Initializing the start index in the machine
start_index = {'machine_4' : 0,
               'machine_5' : 0,
               'machine_6' : 0,
               'machine_7' : 0,
               'machine_8' : 0}


#data_dict = {key : [data_list_X, data_list_Y, prev_Y]}
data_dict = {'machine_4' : [0,0,0],
             'machine_5' : [0,0,0],
             'machine_6' : [0,0,0],
             'machine_7' : [0,0,0],
             'machine_8' : [0,0,0]}

# Reading the Linux process CSV file
input_file = DictReader(open('Streaming_Linux_process.csv'))

# Separating the records in by machines from the file
for row in input_file:
    if int(row['machine']) == 4:
        machine_4.append(row)
    elif int(row['machine']) == 5:
        machine_5.append(row)
    elif int(row['machine']) == 6:
        machine_6.append(row)
    elif int(row['machine']) == 7:
        machine_7.append(row)
    else:
        machine_8.append(row)
        
#print(len(machine_8),len(machine_4),len(machine_5),len(machine_6),len(machine_7))

# Storing the machine records list for each machine key respectively
machine_list = {'machine_4' : machine_4,
                'machine_5' : machine_5,
                'machine_6' : machine_6,
                'machine_7' : machine_7,
                'machine_8' : machine_8}

print('Publishing records..')
while True:
    combined_data = []
    #Looping through the machine list
    for k, value in machine_list.items():
        #Checking for random rows to send
        rows_to_send_X =random.randint(20, 80)
        #Setting the end index for rows to send 
        end_index_X = start_index[k] + rows_to_send_X
        #if 
        #if(end_index_X < len(value)):
        data_dict[k][0] = value[start_index[k] : end_index_X]            
        alter_values(data_dict[k][0])            
        ts = datetime.datetime.utcnow().strftime("%s")            
        add_timestamp(data_dict[k][0],ts)            
        start_index[k] = end_index_X
        #Checks if start_index is greater than value, and sets the start_index to 0
        if(start_index[k] >=len(value)):
            start_index[k]=0
        
        print(k[:9], rows_to_send_X )#+ rows_to_send_Y)
        
    #Sending the data for all the machines in one list of sub-lists containing machine data
    for key in data_dict.keys():
        combined_data.append(data_dict[key][0])
    
    #Publishing the combined data list to the topic
    publish_message(producer, topic,combined_data)
    
    #Reinitializing the combined data list
    combined_data = []
    
    #Assigning sleep for 5 seconds between the published messages
    sleep(5)            
    print('end cycle')